<a href="https://colab.research.google.com/github/jdesous1/MVP-Sprint-1/blob/main/MVP_PUC_RIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MVP PUC RIO: Predição da Quantidade de Alunos a Serem Matriculados em uma Universidade com Base em Dados Históricos

# MVP de Predição da Quantidade de Alunos a Serem Matriculados em uma Universidade com Base em Dados Históricos

## Josafá de Sousa Gomes Júnior

## 1. Definição do Problema


Os dados selecionados (a partir do primeiro semestre de 2014 até o segundo semestre de 2022) para realização da respectiva análise referem-se a um conjunto histórico dos alunos matriculados em uma Instituição de Ensino Superior. Cada linha do dataset fornecido trata-se de um semestre e sua respectiva quantidade de alunos matriculados (**Atributos: Semestre** - Período do ano em que o aluno está matrículado na Universidade  e **IngressosVestibular** - Total de alunos que entraram na Universidade no respectivo período).

A razão pela qual houve esse preocupação está pautada em proporcionar uma melhor experiência ao aluno como melhoria da quantidade de professor por turma (a Universidade possui um padrão de quantidade de docentes por turma), otimização da oferta de disciplinas, enxergar o comportamento para identificar se há padrões, estabelecer planos de ação, reduzir custos, dentre outros. 
Link do dataset escolhido no github: https://raw.githubusercontent.com/jdesous1/MVP-Sprint-1/main/IngressoMatriculados.csv

In [ ]:
# As bibliotecas abaixo foram as importadas para modelagem dos dados e realização de análise dos mesmos

import pandas as pd # utilizada para limpeza, tratamento de dados e análise exploratória de dados
import numpy as np # útil para executar várias tarefas matemáticas como integração numérica, diferenciação, interpolação, extrapolação e etc
import matplotlib # serve para visualização de dados e plotagem gráfica
import matplotlib.pyplot as plt # contém várias funções para criação de gráficos
import seaborn as sns # fornece uma interface de alto nível para criação de gráficos estatísticos atraentes e informativos
import missingno as ms # serve para tratamento de missings
from matplotlib import cm # o "cm" representa "colormap" ou mapa de cores para serem usados em um gráfico
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

## 2. Carga de Dados

A partir do arquivo disponibilizado acima, iremos iniciar algumas análises e visualizações das informações do dataset escolhido afim de explorá-lo e extrair informações suficientemente relevantes.

Para este escopo inicial será utilizado o pacote Pandas para exibição do conjunto de linhas iniciais do dataset escolhido

In [ ]:
# Neste cenário o arquivo csv será importado usando o pacote Pandas através de uma URL

# Declaração da URL na qual os dados serão importados
url = "https://raw.githubusercontent.com/jdesous1/MVP-Sprint-1/main/IngressoMatriculados.csv"

# Iremos agora definir o nome da coluna na qual os dados serão atribuídos
coluna = ['semest','hist']

# Conforme descrito na definição do problema, o período de coleta vai de 2014 à 2022, dessa forma a estrutura ocorrerá conforme segue
dados = pd.read_csv(url, names=coluna, skiprows=1, delimiter=',')

# O objetivo agora é analisar se a estrutura dos dados está de acordo com o que foi definido no passo acima, estruturando a coluna com seu nome e os respectivos valores


In [ ]:
# Com o uso do código dados.head() será possível ter a visão das 5 primeiras linhas do dataframe
dados.head()
# Para a coluna semest parametrizamos que o semestre inicial seria o de número 1 e assim sucessivamente até o preenchimento de todas as informações do dataset.

,semest,hist
0,1,3145
1,2,2458
2,3,3014
3,4,1896
4,5,2532


## 3. Análise de dados

Definida a estrutura do dataset iremos agora realizar inferências sobre o mesmo buscando extrair o máximo de informações dos dados utilizandos os recursos diponibilizados pelas bibliotecas

### 3.1 Dimensão dos dados

Nesta etapa será analisada informações relativas ao dataset como atributos, classe, distribuição dos dados e a variabilidade dos valores.


In [ ]:
# Nesta etapa serão apresentadas detalhes relativos ao dataset
print(dados.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   semest  18 non-null     int64
 1   hist    18 non-null     int64
dtypes: int64(2)
memory usage: 416.0 bytes
None


A partir do processamento realizado acima concluímos que nosso dataset possui 18 valores e todos são não nulos e do tipo inteiro, além disso, como já estabelecido, vemos o nome das duas colunas definidas (semest e hist)